In [1]:
import pybullet
import pybullet_envs
import gym


env = gym.make('Walker2DBulletEnv-v0')
env.render(mode="human")
obs = env.reset()
base_env = env.unwrapped

In [29]:
state = [1,-2,0,0,0,0]
base_env.reset()
for j,s in zip(base_env.ordered_joints, state):
    j.reset_position(s,0)
    
    
env.render()

array([], dtype=float64)

In [28]:
## Mujoco's Walker2d has an action space of Box(17,), bullet has Box(22,) unclear what the extra states are

obs, reward, done, _ = env.step(env.action_space.sample())
print(obs)
env.render()

[-8.7521307e-04  0.0000000e+00  1.0000000e+00 -7.8132980e-02
  0.0000000e+00 -4.6093658e-02  0.0000000e+00 -9.3338070e-03
  1.7496320e+00 -1.8118952e-01 -5.2626938e-01  1.9968210e-02
  9.3382550e-03  7.1170345e-02  1.0000000e+00  0.0000000e+00
  9.7172242e-01 -3.5873953e-01  2.8301427e-02  2.1311554e-01
  0.0000000e+00  0.0000000e+00]


array([], dtype=float64)